In [ ]:
import requests
import json
import itertools
import socks
import socket
import logging
import pandas as pd
import sys

from bs4 import BeautifulSoup
from fake_useragent import UserAgent

In [ ]:
logging.basicConfig(stream=sys.stdout)
logger = logging.Logger('parser')
logger.setLevel(logging.DEBUG)
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
socket.socket = socks.socksocket

In [ ]:
UserAgent().chrome

In [ ]:
class Parser:
    def __init__(self,base_url='https://api.superjob.ru/2.0):
        time.sleep(1)
        self.base_url = base_url
        self.parsed_vacancies = []
    
    def get_areas(self):
        res = []
        for i in json.loads(requests.get(self.base_url + 'towns').text)[0]['towns']:
            res.append((i['id'], i['name']))
        return res
    
    def get_vacancies(self):
        self.parsed_vacancies = []
        experience = ['1', '2', '3', '4']
        for area, area_name in self.get_areas():
            logger.warning('current area: {}'.format(area_name))
            for exp, ind in itertools.product(experience, [i for i in range(100)]):
                for i in range(20):
                    time.sleep(60)
                    r = requests.get(self.base_url + 'vacancies', 
                                     params={'text': '', 'per_page': 100, 'page': i, 'area': area, 'industry': ind, 'experience': exp},
                                     headers={'User-Agent': UserAgent().chrome})
                    if not r.ok:
                        logger.warning('page error resp code: {}'.format(r.status_code))
                        logger.warning(r.text)
                    parsed = json.loads(r.text)
                    if parsed.get('items'):
                        self.parsed_vacancies.extend(parsed.get('items'))
                        logger.warning('parsed: {}'.format(len(parsed.get('items'))))
                    else:
                        break
                    if len(parsed.get('items')) < 100:
                        break
        return self.parsed_vacancies
    
    def to_pandas(self):
        conv = []
        for vacancy in self.parsed_vacancies:
            conv.append(
                [vacancy['town']['name'],
                 vacancy['firm_name'],
                 vacancy['salary'],
                 vacancy['name'],
                 vacancy['snippet'].get('requirement', ''),
                 vacancy['snippet'].get('responsibility', '')]
            )
        return pd.DataFrame(data=conv, columns=['area', 'company_name', 'salary', 'name', 'req', 'resp'])

